In [1]:
from src import util
import tensorflow as tf
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sensor_data_trainval, sequence_ids_trainval, labels_trainval = util.ingest_sensor_data('train.csv', 'train_labels.csv')
sensor_data_test, sequence_ids_test, _ = util.ingest_sensor_data('test.csv')

sensor_data_train, sequence_ids_train, labels_train, sensor_data_val, sequence_ids_val, labels_val = util.validation_split(training_fraction=0.7, sensor_data=sensor_data_trainval, sequence_ids=sequence_ids_trainval, labels=labels_trainval)

(25968, 60, 13) 25968
(12218, 60, 13) 12218


In [ ]:
lstm_dim = 128
dense_dim=128
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout, recurrent_dropout=lstm_dropout)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50)
pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'lstm-bidirectional')

In [ ]:
util.generate_submission(pred, sequence_ids_test, 'lstm')

In [ ]:
lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout, recurrent_dropout=0),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50)
pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'lstm-unidirectional-64')

In [4]:
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# additional layer in the classifier dense net does not help
lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout, recurrent_dropout=0),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50)
pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'lstm-unidirectional-64-2layerhead')

In [5]:
# additional lstm layer
lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout, recurrent_dropout=0,return_sequences=True),
    tf.keras.layers.LSTM(units=lstm_dim, dropout=lstm_dropout),

    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50)
pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'lstm-unidirectional-64-2layer')
# maybe slightly helps


Epoch 1/50
650/650 [==============================] - 17s 18ms/step - loss: 0.6870 - accuracy: 0.5466 - auc: 0.5620 - val_loss: 0.6806 - val_accuracy: 0.5841 - val_auc: 0.6139
Epoch 2/50
650/650 [==============================] - 11s 17ms/step - loss: 0.6747 - accuracy: 0.5800 - auc: 0.6083 - val_loss: 0.6649 - val_accuracy: 0.6190 - val_auc: 0.6489
Epoch 3/50
650/650 [==============================] - 11s 17ms/step - loss: 0.6524 - accuracy: 0.6260 - auc: 0.6585 - val_loss: 0.6327 - val_accuracy: 0.6542 - val_auc: 0.6998
Epoch 4/50
650/650 [==============================] - 11s 17ms/step - loss: 0.6292 - accuracy: 0.6485 - auc: 0.6971 - val_loss: 0.6175 - val_accuracy: 0.6702 - val_auc: 0.7512
Epoch 5/50
650/650 [==============================] - 11s 17ms/step - loss: 0.5686 - accuracy: 0.7088 - auc: 0.7771 - val_loss: 0.5116 - val_accuracy: 0.7620 - val_auc: 0.8316
Epoch 6/50
650/650 [==============================] - 10s 16ms/step - loss: 0.5286 - accuracy: 0.7358 - auc: 0.8132 - va

KeyboardInterrupt: 

In [8]:
# conv input to lstm layer lstm_dim = 64 (to see if GPU helps)
dense_dim=64
lstm_dropout=0.15
filters=128
kernel_size=5

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(0.15),
   
    tf.keras.layers.LSTM(units=lstm_dim, dropout=lstm_dropout),
    
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50)
pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d-lstm')
# maybe slightly helps


Epoch 1/50
650/650 [==============================] - 10s 13ms/step - loss: 0.6690 - accuracy: 0.5860 - auc: 0.6155 - val_loss: 0.6435 - val_accuracy: 0.6309 - val_auc: 0.6659
Epoch 2/50
650/650 [==============================] - 7s 11ms/step - loss: 0.6318 - accuracy: 0.6412 - auc: 0.6759 - val_loss: 0.6117 - val_accuracy: 0.6571 - val_auc: 0.7203
Epoch 3/50
650/650 [==============================] - 7s 11ms/step - loss: 0.6022 - accuracy: 0.6726 - auc: 0.7222 - val_loss: 0.6020 - val_accuracy: 0.6660 - val_auc: 0.7590
Epoch 4/50
650/650 [==============================] - 7s 11ms/step - loss: 0.5719 - accuracy: 0.6998 - auc: 0.7626 - val_loss: 0.4991 - val_accuracy: 0.7528 - val_auc: 0.8328
Epoch 5/50
650/650 [==============================] - 8s 12ms/step - loss: 0.4809 - accuracy: 0.7646 - auc: 0.8494 - val_loss: 0.4035 - val_accuracy: 0.8094 - val_auc: 0.8994
Epoch 6/50
650/650 [==============================] - 7s 11ms/step - loss: 0.4115 - accuracy: 0.8108 - auc: 0.8937 - val_los

In [9]:
# conv input to lstm layer lstm_dim = 64 (to see if GPU helps)
dense_dim=64
lstm_dropout=0.15
filters=128
kernel_size=5

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(0.15),
   
    tf.keras.layers.LSTM(units=lstm_dim, dropout=lstm_dropout),
    
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(sensor_data_trainval, np.array(labels_trainval), validation_split=0.2, epochs=50)
pred = model.predict(sensor_data_test)
util.generate_submission(pred, sequence_ids_test, 'conv1d-lstm-raw')
# maybe slightly helps


Epoch 1/50
650/650 [==============================] - 11s 15ms/step - loss: 0.6778 - accuracy: 0.5675 - auc: 0.5941 - val_loss: 0.6526 - val_accuracy: 0.6113 - val_auc: 0.6466
Epoch 2/50
650/650 [==============================] - 8s 12ms/step - loss: 0.6438 - accuracy: 0.6295 - auc: 0.6700 - val_loss: 0.6293 - val_accuracy: 0.6392 - val_auc: 0.6958
Epoch 3/50
650/650 [==============================] - 8s 12ms/step - loss: 0.6173 - accuracy: 0.6613 - auc: 0.7117 - val_loss: 0.5862 - val_accuracy: 0.6910 - val_auc: 0.7507
Epoch 4/50
650/650 [==============================] - 8s 12ms/step - loss: 0.5747 - accuracy: 0.7025 - auc: 0.7632 - val_loss: 0.5427 - val_accuracy: 0.7187 - val_auc: 0.7972
Epoch 5/50
650/650 [==============================] - 8s 12ms/step - loss: 0.5330 - accuracy: 0.7360 - auc: 0.8057 - val_loss: 0.5057 - val_accuracy: 0.7522 - val_auc: 0.8288
Epoch 6/50
650/650 [==============================] - 8s 12ms/step - loss: 0.4912 - accuracy: 0.7596 - auc: 0.8395 - val_los